In [ ]:
'''
This notebook is based on the following kaggle notebook:
https://www.kaggle.com/yuanquan/covid-19-prediction-by-country-and-province
'''
import numpy as np
import pandas as pd
import datetime

# hide warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

In [ ]:
# load global data
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

confirmed_table = confirmed_df.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], var_name="Date", value_name="Confirmed").fillna('').drop(['Lat', 'Long'], axis=1)
death_table = death_df.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], var_name="Date", value_name="Deaths").fillna('').drop(['Lat', 'Long'], axis=1)
recovered_table = recovered_df.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], var_name="Date", value_name="Recovered").fillna('').drop(['Lat', 'Long'], axis=1)

# merge into 1 table
full_table = confirmed_table.merge(death_table).merge(recovered_table)
full_table['Date'] = pd.to_datetime(full_table['Date'])

# Add active
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

# replacing Mainland china with just China
full_table['Country/Region'] = full_table['Country/Region'].replace('Mainland China', 'China')

# filling missing values 
full_table[['Province/State']] = full_table[['Province/State']].fillna('')

In [ ]:
def get_time_series(country):
    # for some countries, data is spread over several Provinces
    if full_table[full_table['Country/Region'] == country]['Province/State'].nunique() > 1:
        country_table = full_table[full_table['Country/Region'] == country]
        country_df = pd.DataFrame(pd.pivot_table(country_table, values = ['Confirmed', 'Deaths', 'Recovered', 'Active'],
                              index='Date', aggfunc=sum).to_records())
        return country_df.set_index('Date')[['Confirmed', 'Deaths', 'Recovered', 'Active']]
    df = full_table[(full_table['Country/Region'] == country) 
                & (full_table['Province/State'].isin(['', country]))]
    return df.set_index('Date')[['Confirmed', 'Deaths', 'Recovered', 'Active']]

def get_time_series_province(province):
    df = full_table[(full_table['Province/State'] == province)]
    return df.set_index('Date')[['Confirmed', 'Deaths', 'Recovered', 'Active']]

def gen_xy_from_data(data, in_size, out_size):
    num_xy = data.size - in_size - out_size + 1
    y = np.column_stack(data[i:i+num_xy] for i in range(in_size, in_size+out_size))
    x = np.column_stack(data[i:i+num_xy] for i in range(in_size))
    
    input_normalization = 100
    x /= input_normalization
    y /= input_normalization

    return x, y

In [ ]:
country_table = full_table[full_table['Country/Region'] == "China"]
country_table.Active[country_table.Active < 0] = 0
provinces = country_table["Province/State"].unique()

In [ ]:
in_size = 16
out_size = 16

In [ ]:
x = np.zeros((0, in_size))
y = np.zeros((0, out_size))
for prov in provinces:
    if prov == "Hubei":
        continue
    prov_df = country_table[country_table["Province/State"] == prov]
    active = prov_df.values[:, -1]
    x_prov, y_prov = gen_xy_from_data(active, in_size, out_size)
    x = np.row_stack((x, x_prov))
    y = np.concatenate((y, y_prov))

In [ ]:
np.save("x", x)
np.save("y", y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xt, xv, yt, yv = train_test_split(x, y, test_size=0.3, random_state = 2020)
np.save("x_train", xt)
np.save("y_train", yt)
np.save("x_val", xv)
np.save("y_val", yv)